### MOT metrics

In [2]:
!pip install motmetrics

In [2]:
import numpy as np
import glob
import os
import pandas as pd
import logging
from collections import OrderedDict
from pathlib import Path
import motmetrics as mm

In [3]:
# List all default metrics
mh = mm.metrics.create()
print(mh.list_metrics_markdown())

Name|Description
:---|:---
num_frames|Total number of frames.
obj_frequencies|Total number of occurrences of individual objects over all frames.
pred_frequencies|Total number of occurrences of individual predictions over all frames.
num_matches|Total number matches.
num_switches|Total number of track switches.
num_transfer|Total number of track transfer.
num_ascend|Total number of track ascend.
num_migrate|Total number of track migrate.
num_false_positives|Total number of false positives (false-alarms).
num_misses|Total number of misses.
num_detections|Total number of detected objects including matches and switches.
num_objects|Total number of unique object appearances over all frames.
num_predictions|Total number of unique prediction appearances over all frames.
num_unique_objects|Total number of unique object ids encountered.
track_ratios|Ratio of assigned to total appearance count per unique object id.
mostly_tracked|Number of objects tracked for at least 80 percent of lifespan.
par

### Evaluate MOT metrics

Functions to compute motchallenge metrics

In [4]:
def compare_dataframes(gts, ts):
    accs = []
    names = []
    for k, tsacc in ts.items():
        if k in gts:
            logging.info('Comparing {}...'.format(k))
            accs.append(mm.utils.compare_to_groundtruth(gts[k], tsacc, 'iou', distth=0.5))
            names.append(k)
        else:
            logging.warning('No ground truth for {}, skipping.'.format(k))

    return accs, names


def compute_motmetrics(groundtruths, tests, score_threshold, fmt='mot15-2D'):

    gtfiles = glob.glob(os.path.join(groundtruths, '*/gt/gt.txt'))
    print('gt_files', gtfiles)
    tsfiles = [f for f in glob.glob(os.path.join(tests, '*.txt')) if not os.path.basename(f).startswith('eval')]

    print('Found {} groundtruths and {} test files.'.format(len(gtfiles), len(tsfiles)))
    print('Available LAP solvers {}'.format(mm.lap.available_solvers))
    print('Default LAP solver \'{}\''.format(mm.lap.default_solver))
    print('Loading files.')

    gt = OrderedDict([(Path(f).parts[-3], mm.io.loadtxt(f, fmt=fmt, min_confidence=1)) for f in gtfiles])
    ts = OrderedDict(
        [(os.path.splitext(Path(f).parts[-1])[0], mm.io.loadtxt(f, fmt=fmt, min_confidence=score_threshold))
         for f in tsfiles])
    #     ts = gt
    mh = mm.metrics.create()
    accs, names = compare_dataframes(gt, ts)

    logging.info('Running metrics')
    metrics = ['recall', 'precision', 'num_unique_objects', 'mostly_tracked',
               'partially_tracked', 'mostly_lost', 'num_false_positives', 'num_misses',
               'num_switches', 'num_fragmentations', 'mota', 'motp', 'idf1', 'num_objects']
    summary = mh.compute_many(accs, names=names, metrics=metrics, generate_overall=True)
    div_dict = {
        'num_objects': ['num_false_positives', 'num_misses', 'num_switches', 'num_fragmentations'],
        'num_unique_objects': ['mostly_tracked', 'partially_tracked', 'mostly_lost']}
    for divisor in div_dict:
        for divided in div_dict[divisor]:
            summary[divided] = (summary[divided] / summary[divisor])
    fmt = mh.formatters
    change_fmt_list = ['num_false_positives', 'num_misses', 'num_switches', 'num_fragmentations', 'mostly_tracked',
                       'partially_tracked', 'mostly_lost']
    for k in change_fmt_list:
        fmt[k] = fmt['mota']
    print(mm.io.render_summary(summary, formatters=fmt, namemap=mm.io.motchallenge_metric_names))


### Calculate metrics for DeepSort MOT20

GT - directory with ground truth sequences.
DeepSort_predictions - directory with DeepSort predictions (*.txt files)

In [5]:
compute_motmetrics(groundtruths="GT", tests="results", score_threshold=0.8, fmt='mot15-2D')

gt_files ['GT/MOT20-03/gt/gt.txt', 'GT/MOT20-02/gt/gt.txt', 'GT/MOT20-05/gt/gt.txt', 'GT/MOT20-01/gt/gt.txt']
Found 4 groundtruths and 5 test files.
Available LAP solvers ['scipy']
Default LAP solver 'scipy'
Loading files.
OrderedDict([('MOT20-03',                 X    Y  Width  Height  Confidence  ClassId  Visibility
FrameId Id                                                            
695     1    1126  816     45      62           1        1     1.00000
696     1    1123  814     47      64           1        1     1.00000
697     1    1121  812     48      66           1        1     1.00000
698     1    1119  810     49      68           1        1     1.00000
699     1    1117  808     50      70           1        1     1.00000
...           ...  ...    ...     ...         ...      ...         ...
2144    733   338    0     37      18           1        1     0.74792
2145    733   338    0     37      16           1        1     0.72136
2146    733   338    0     37      14    

         Rcll Prcn   GT   MT   PT     ML      FP     FN  IDs   FM     MOTA  MOTP IDF1 num_objects
MOT20-01 0.0% 0.0%   74 0.0% 0.0% 100.0% 1611.5% 100.0% 0.0% 0.0% -1611.5%   NaN 0.0%       19870
MOT20-03 0.0% 0.1%  702 0.0% 0.0% 100.0%   44.0% 100.0% 0.0% 0.0%   -44.0% 0.429 0.0%      313658
MOT20-02 0.0% 0.0%  270 0.0% 0.0% 100.0%   48.8% 100.0% 0.0% 0.0%   -48.8%   NaN 0.0%      154742
OVERALL  0.0% 0.0% 1046 0.0% 0.0% 100.0%  109.3% 100.0% 0.0% 0.0%  -109.3%   NaN 0.0%      488270
